# Quantifying Uncertainty
This notebook serves as the supporting material for Chapter 13 **Quantifying Uncertainty**. In this notebook we will show how to use the code repository to tame unceratinty with degrees of belief. In this notebook we will concentrate on the task of representing and computing with probabilistic information in general. First, let's load the aima-jar.

In [2]:
%classpath add jar ../out/artifacts/aima_core_jar/aima-core.jar

## Basic Probability Notion
For our agent to represent and use probabilistic information, we need a formal language. Appendix A(from text) includes a standard introduction to
elementary probability theory. Here, we take an approach more suited to the needs of AI and
more consistent with the concepts of formal logic.

In probability
theory, the set of all possible worlds is called the **sample space**. The possible worlds are
mutually exclusive and exhaustive—two possible worlds cannot both be the case, and one
possible world must be the case. For example, if we are about to roll two (distinguishable)
dice, there are $36$ possible worlds to consider: $(1,1), (1,2), . . ., (6,6)$. The Greek letter $\Omega$
(uppercase omega) is used to refer to the sample space, and $\omega$ (lowercase omega) refers to
elements of the space, that is, particular possible worlds.

A fully specified **probability model** associates a numerical probability $P (\omega)$ with each
possible world.The basic axioms of probability theory say that every possible world has a
probability between 0 and 1 and that the total probability of the set of possible worlds is 1

$0 \leq P (\omega) \leq 1$ for every $\omega$ and $\displaystyle\sum_{\omega \in \Omega}P (\omega) = 1$

For example, if we assume that each die is fair and the rolls don’t interfere with each other,
then each of the possible worlds (1,1), (1,2), . . ., (6,6) has probability $frac{1}{36}$. On the other
hand, if the dice conspire to produce the same number, then the worlds (1,1), (2,2), (3,3), etc.,
might have higher probabilities, leaving the others with lower probabilities.

Probabilistic assertions and queries are not usually about particular possible worlds, but
about sets of them.In AI,
the sets are always described by propositions in a formal language.Propositions describing sets of possible worlds are written in a notation that combines elements of propositional logic and constraint satisfaction notation.The probability associated with a proposition is defined to be the sum of the probabilities of the worlds in which it holds

For any proposition, $\phi, P(\phi) = \displaystyle\sum_{\omega in \phi} P(\omega)$.

The code repository already contains a variety of propositions in the [proposition module](https://github.com/aimacode/aima-java/tree/AIMA3e/aima-core/src/main/java/aima/core/probability/proposition). However, to implement a custom proposition, we can always extend the Proposition interface which is as follows :

````java

public interface Proposition {
	/**
	 * 
	 * @return the Set of RandomVariables in the World (sample space) that this
	 *         Proposition is applicable to.
	 */
	Set<RandomVariable> getScope();

	/**
	 * 
	 * @return the Set of RandomVariables from this propositions scope that are
	 *         not constrained to any particular set of values (e.g. bound =
	 *         P(Total = 11), while unbound = P(Total)). If a variable is
	 *         unbound it implies the distributions associated with the variable
	 *         is being sought.
	 */
	Set<RandomVariable> getUnboundScope();

	/**
	 * Determine whether or not the proposition holds in a particular possible
	 * world.
	 * 
	 * @param possibleWorld
	 *            A possible world is defined to be an assignment of values to
	 *            all of the random variables under consideration.
	 * @return true if the proposition holds in the given possible world, false
	 *         otherwise.
	 */
	boolean holds(Map<RandomVariable, Object> possibleWorld);
}
````

Variables in probability theory are called **random variables** and their names begin with
an uppercase letter. Every random variable has a domain. For eg, in the random event of throwing two dice simultaneously, let $Total$ represent the sum of the numbers on the dice and let $Die_1$ represent the number on the first die. Thus, in the dice example, Total and Die 1 are random variables. The domain of
Total for two dice is the set {2, . . . , 12} and the domain of Die 1 is {1, . . . , 6}. The code repository already contains a lot of implemented domains and random variables, but let us create our own custom random variable and a custom domain for the dice example above
We will first implement a domain for the dice example using AbstractFiniteDomain class which is an extra layer on our Domain interface.



In [3]:
package aima.notebooks.quantifyinguncertainty;

import aima.core.probability.domain.*;
import java.util.*;

class TotalDiceDomain extends AbstractFiniteDomain{
    
    Set<Integer> values = new HashSet<>();
    
    public TotalDiceDomain(){
        for(int i =2;i<=12;i++){
            values.add(i);
        }
    }
    
    
    @Override
    public int size(){
        return values.size();
    }
    
    @Override
    public boolean isOrdered(){
        return true;
    }
    
    @Override
    public Set<Integer> getPossibleValues(){
        return values;
    }
}

aima.notebooks.quantifyinguncertainty.TotalDiceDomain

Let us look how to create a Probability Model. To create a ProbabilityModel, we have to implement the ProbabilityModel interface from the code. The following is the ProbabilityModelInterface :

````java
public interface ProbabilityModel {
	/**
	 * The default threshold for rounding errors. Example, to test if
	 * probabilities sum to 1:<br>
	 * <br>
	 * Math.abs(1 - probabilitySum) <
	 * ProbabilityModel.DEFAULT_ROUNDING_THRESHOLD;
	 */
	final double DEFAULT_ROUNDING_THRESHOLD = 1e-8;

	/**
	 * 
	 * @return true, if 0 <= P(&omega;) <= 1 for every &omega; and
	 *         &sum;<sub>&omega; &isin; &Omega;</sub> P(&omega;) = 1 (Equation
	 *         13.1 pg. 484 AIMA3e), false otherwise.
	 */
	boolean isValid();

	/**
	 * For any proposition &phi;, P(&phi;) = &sum;<sub>&omega; &isin;
	 * &phi;</sub> P(&omega;). Refer to equation 13.2 page 485 of AIMA3e.
	 * Probabilities such as P(Total = 11) and P(doubles) are called
	 * unconditional or prior probabilities (and sometimes just "priors" for
	 * short); they refer to degrees of belief in propositions in the absence of
	 * any other information.
	 * 
	 * @param phi
	 *            the propositional terms for which a probability value is to be
	 *            returned.
	 * @return the probability of the proposition &phi;.
	 */
	double prior(Proposition... phi);

	/**
	 * Unlike unconditional or prior probabilities, most of the time we have
	 * some information, usually called evidence, that has already been
	 * revealed. This is the conditional or posterior probability (or just
	 * "posterior" for short). Mathematically speaking, conditional
	 * probabilities are defined in terms of unconditional probabilities as
	 * follows, for any propositions a and b, we have:<br>
	 * <br>
	 * P(a | b) = P(a AND b)/P(b)<br>
	 * <br>
	 * which holds whenever P(b) > 0. Refer to equation 13.3 page 485 of AIMA3e.
	 * This can be rewritten in a different form called the <b>product rule</b>: <br>
	 * <br>
	 * P(a AND b) = P(a | b)P(b)<br>
	 * <br>
	 * and also as:<br>
	 * <br>
	 * P(a AND b) = P(b | a)P(a)<br>
	 * <br>
	 * whereby, equating the two right-hand sides and dividing by P(a) gives you
	 * Bayes' rule:<br>
	 * <br>
	 * P(b | a) = (P(a | b)P(b))/P(a) - i.e. (likelihood * prior)/evidence
	 * 
	 * @param phi
	 *            the proposition for which a probability value is to be
	 *            returned.
	 * @param evidence
	 *            information we already have.
	 * @return the probability of the proposition &phi; given evidence.
	 */
	double posterior(Proposition phi, Proposition... evidence);

	/**
	 * @return a consistent ordered Set (e.g. LinkedHashSet) of the random
	 *         variables describing the atomic variable/value pairs this
	 *         probability model can take on. Refer to pg. 486 AIMA3e.
	 */
	Set<RandomVariable> getRepresentation();
````